In [1]:
import numpy as np, glob, librosa
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2   

# next cell
%reload_ext autoreload

In [2]:
files = sorted(glob.glob("./data/fma-small/*/*.mp3"))

In [3]:
sampling=16000
file_data, sr = librosa.load(files[0], sr=sampling, mono=True)

/home/rook/.local/share/virtualenvs/csci699-project-sCaErgvQ/lib/python3.7/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


In [4]:
import tensorflow as tf
print(tf.__version__)
import wavenet_tf as tf_wn

1.15.0


In [5]:
net = tf_wn.WaveNetModel(
        batch_size=32,
        dilations=[1, 2, 4, 8, 16, 32, 64, 128, 256, 512,
                   1, 2, 4, 8, 16, 32, 64, 128, 256, 512,
                   1, 2, 4, 8, 16, 32, 64, 128, 256, 512,
                   1, 2, 4, 8, 16, 32, 64, 128, 256, 512,
                   1, 2, 4, 8, 16, 32, 64, 128, 256, 512],
        filter_width=2,
        residual_channels=32,
        dilation_channels=32,
        skip_channels=512,
        quantization_channels=256,
        use_biases=True,
        scalar_input=False,
        initial_filter_width=32,
        histograms=False,
        global_condition_channels=None,
        global_condition_cardinality=None)

In [ ]:
net
